In [1]:
import pandas as pd 
import numpy as np
import pandas as pd
import os
import json
from tqdm import tqdm

In [2]:
games_users_df = pd.read_csv('../dataset/weighted_score_above_08.csv')

C:\Users\sbm76\AppData\Local\Temp\ipykernel_10884\3570771008.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  games_users_df = pd.read_csv('../dataset/weighted_score_above_08.csv')


In [3]:
games_users_df

,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,1696305457,...,1,99,2,0.889438,0,1,0,0,1,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,1696096555,...,1,122,8,0.914834,0,1,0,0,1,NaN
2,147357703,10,Counter-Strike,76561199080026894,118,23,13501,212,12957,1697630734,...,1,599,20,0.968375,3,1,0,0,1,NaN
3,147345102,10,Counter-Strike,76561198068970227,28,1,10668,640,9906,1698261011,...,1,59,12,0.826206,0,0,0,0,1,NaN
4,147284743,10,Counter-Strike,76561199137893460,19,5,543,10,526,1697810991,...,1,128,24,0.853612,0,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498089,93910882,997010,Police Simulator: Patrol Officers,76561198055740634,0,15,22402,420,103,1698784566,...,1,168,13,0.857843,8,1,0,1,0,NaN
498090,93909129,997010,Police Simulator: Patrol Officers,76561198107142741,0,69,18,0,18,1623946025,...,1,1417,1226,0.938828,30,1,0,1,0,NaN
498091,51109633,997060,Kotodama: The 7 Mysteries of Fujisawa,76561198091884692,1588,32,835,0,835,1559864576,...,1,66,14,0.847317,0,0,1,0,0,NaN
498092,48814832,997180,be you,76561198136664035,95,8,7,0,7,1549516192,...,1,57,7,0.838971,2,0,0,0,0,NaN


In [3]:
games_users_df.shape

(498094, 24)

In [4]:
games_users_df['appid'].nunique()

27315

In [5]:
games_users_df['author_steamid'].nunique()

379636

Create the User to Game Mapping Dictionary.

In [15]:
user2game = games_users_df.groupby('author_steamid')['appid'].apply(list).to_dict()

Create the Game to User Mapping Dictionary.

In [16]:
game2user = games_users_df.groupby('appid')['author_steamid'].apply(list).to_dict()

Create the (User, Game) to Rating Dictionary.

In [17]:
user_game = zip(games_users_df['author_steamid'], games_users_df['appid'])
user_game_rating = zip(user_game, games_users_df['voted_up'])
usergame2rating = dict(user_game_rating)

## ALS via the Implicit Library

In [13]:
import numpy as np
import scipy.sparse as sp
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k
from sklearn.metrics import mean_squared_error
import itertools

In [14]:
# Convert explicit binary feedback to implicit-style confidence matrix
# -------------------------------------------------------------------
# Create mappings
users = list(user2game.keys())
games = list(game2user.keys())
user_id_map = {u: i for i, u in enumerate(users)}
game_id_map = {g: i for i, g in enumerate(games)}

# Build sparse matrix with confidence weights
rows, cols, data = [], [], []
for (user, game), rating in usergame2rating.items():
    rows.append(user_id_map[user])
    cols.append(game_id_map[game])
    # Convert explicit 0/1 to confidence scores
    data.append(1.0 if rating == 1 else 0.1)  # 1.0 for likes, 0.1 for dislikes

matrix = sp.csr_matrix((data, (rows, cols)), shape=(len(users), len(games)))

# Split data into train/test (80/20)
train, test = train_test_split(matrix, train_percentage=0.8, random_state=42)

In [15]:
# Hyperparameter Tuning Setup
# ---------------------------
param_grid = {
    'factors': [20, 40, 60],  # Latent features
    'regularization': [0.01, 0.1, 1.0]  # L2 regularization
}

best_score = float('inf')
best_params = {}
results = []

In [16]:
# Custom Evaluation Functions
# ---------------------------
def calculate_rmse(model, test_data):
    """Calculate RMSE for explicit binary feedback"""
    rows, cols = test_data.nonzero()
    preds = []
    actuals = []
    for row, col in zip(rows, cols):
        pred = model.user_factors[row] @ model.item_factors[col].T
        actual = test_data[row, col]
        preds.append(pred)
        actuals.append(actual)
    return np.sqrt(mean_squared_error(actuals, preds))

def recall_at_k(model, train_matrix, test_matrix, K=10):
    """Custom implementation of recall@k for implicit library"""
    test_coo = test_matrix.tocoo()
    user_test_items = {}
    for user, item in zip(test_coo.row, test_coo.col):
        user_test_items.setdefault(user, set()).add(item)
    
    total_recall = 0.0
    num_users = 0
    
    for user in user_test_items:
        # Get top-K recommendations excluding training items
        recommended, _ = model.recommend(
            userid=user,
            user_items=train_matrix[user],
            N=K,
            filter_items=train_matrix[user].indices
        )
        
        relevant = user_test_items[user]
        if len(relevant) > 0:
            hits = len(set(recommended) & relevant)
            total_recall += hits / len(relevant)
            num_users += 1
    
    return total_recall / num_users if num_users > 0 else 0.0

In [17]:
# Hyperparameter Search
# ---------------------
for factors, reg in itertools.product(param_grid['factors'], param_grid['regularization']):
    # Initialize and train model
    model = AlternatingLeastSquares(
        factors=factors,
        regularization=reg,
        iterations=15,
        use_gpu=False,
        random_state=42
    )
    
    # Convert confidence matrix to COO format for training
    train_coo = train.tocoo()
    model.fit(train_coo)
    
    # Calculate metrics
    rmse = calculate_rmse(model, test)
    prec = precision_at_k(model, train, test.T.tocsr(), K=10)
    rec = recall_at_k(model, train, test)
    
    results.append({
        'factors': factors,
        'regularization': reg,
        'rmse': rmse,
        'precision@10': prec,
        'recall@10': rec
    })
    
    # Track best model based on RMSE
    if rmse < best_score:
        best_score = rmse
        best_params = {'factors': factors, 'regularization': reg}
        best_model = model

c:\Users\sbm76\anaconda3\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 6 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
c:\Users\sbm76\anaconda3\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.003998756408691406 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/14012 [00:00<?, ?it/s]

c:\Users\sbm76\anaconda3\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0029838085174560547 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/14012 [00:00<?, ?it/s]

c:\Users\sbm76\anaconda3\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0029990673065185547 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/14012 [00:00<?, ?it/s]

In [18]:
# Print results
print("Hyperparameter Tuning Results:")
for res in results:
    print(f"Factors: {res['factors']}, Reg: {res['regularization']}: "
          f"RMSE: {res['rmse']:.4f}, Precision@10: {res['precision@10']:.4f}, "
          f"Recall@10: {res['recall@10']:.4f}")

print(f"\nBest Parameters: {best_params} with RMSE: {best_score:.4f}")

Hyperparameter Tuning Results:
Factors: 20, Reg: 0.01: RMSE: 0.9185, Precision@10: 0.0001, Recall@10: 0.0081
Factors: 20, Reg: 0.1: RMSE: 0.9186, Precision@10: 0.0001, Recall@10: 0.0081
Factors: 20, Reg: 1.0: RMSE: 0.9186, Precision@10: 0.0001, Recall@10: 0.0081
Factors: 40, Reg: 0.01: RMSE: 0.9184, Precision@10: 0.0000, Recall@10: 0.0075
Factors: 40, Reg: 0.1: RMSE: 0.9185, Precision@10: 0.0001, Recall@10: 0.0074
Factors: 40, Reg: 1.0: RMSE: 0.9185, Precision@10: 0.0001, Recall@10: 0.0076
Factors: 60, Reg: 0.01: RMSE: 0.9183, Precision@10: 0.0001, Recall@10: 0.0079
Factors: 60, Reg: 0.1: RMSE: 0.9183, Precision@10: 0.0001, Recall@10: 0.0079
Factors: 60, Reg: 1.0: RMSE: 0.9184, Precision@10: 0.0001, Recall@10: 0.0078

Best Parameters: {'factors': 60, 'regularization': 0.01} with RMSE: 0.9183


In [19]:
# Recommendation Function
# -----------------------
def recommend_for_user(user_id, liked_games, disliked_games, k=10):
    """Generate recommendations for a new user"""
    # Create user vector
    user_vec = sp.lil_matrix((1, len(games)))
    for g in liked_games:
        if g in game_id_map:
            user_vec[0, game_id_map[g]] = 1.0  # High confidence for likes
    for g in disliked_games:
        if g in game_id_map:
            user_vec[0, game_id_map[g]] = 0.1  # Low confidence for dislikes
    
    # Generate recommendations
    ids, scores = best_model.recommend(
        userid=0,
        user_items=user_vec.tocsr(),
        filter_items=[game_id_map[g] for g in liked_games + disliked_games],
        N=k
    )
    
    # Map back to original game IDs
    reverse_game_map = {v: k for k, v in game_id_map.items()}
    return [(reverse_game_map[i], s) for i, s in zip(ids, scores)]

In [20]:
# Example usage:
user_likes = [10, 1002560]  # Replace with actual appids
user_dislikes = [10090]       # Replace with actual appids
recommendations = recommend_for_user("new_user", user_likes, user_dislikes)

In [21]:
recommendations

[(22380, 0.026209066),
 (12210, 0.013534127),
 (620, 0.011088431),
 (238320, 0.010560356),
 (550, 0.008859815),
 (240, 0.008714085),
 (379720, 0.008494401),
 (632360, 0.008273686),
 (319630, 0.007957619),
 (3590, 0.0075515066)]

## Memory Based Item Based Collaborative Filtering

In [3]:
from itertools import combinations
import pickle

In [4]:
games_users_df

,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,1696305457,...,1,99,2,0.889438,0,1,0,0,1,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,1696096555,...,1,122,8,0.914834,0,1,0,0,1,NaN
2,147357703,10,Counter-Strike,76561199080026894,118,23,13501,212,12957,1697630734,...,1,599,20,0.968375,3,1,0,0,1,NaN
3,147345102,10,Counter-Strike,76561198068970227,28,1,10668,640,9906,1698261011,...,1,59,12,0.826206,0,0,0,0,1,NaN
4,147284743,10,Counter-Strike,76561199137893460,19,5,543,10,526,1697810991,...,1,128,24,0.853612,0,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498089,93910882,997010,Police Simulator: Patrol Officers,76561198055740634,0,15,22402,420,103,1698784566,...,1,168,13,0.857843,8,1,0,1,0,NaN
498090,93909129,997010,Police Simulator: Patrol Officers,76561198107142741,0,69,18,0,18,1623946025,...,1,1417,1226,0.938828,30,1,0,1,0,NaN
498091,51109633,997060,Kotodama: The 7 Mysteries of Fujisawa,76561198091884692,1588,32,835,0,835,1559864576,...,1,66,14,0.847317,0,0,1,0,0,NaN
498092,48814832,997180,be you,76561198136664035,95,8,7,0,7,1549516192,...,1,57,7,0.838971,2,0,0,0,0,NaN


In [18]:
# Create dictionary: {appid: set of users who rated it}
game_users = {game: set(users) for game, users in game2user.items()}

In [22]:
# Dictionaries are already defined so let's calculate similarity between items using Jaccard Similarity

# Calculate Jaccard similarity for item pairs with ≥5 common users
item_similarities = {}
for (game1, game2) in combinations(game_users.keys(), 2):
    common_users = game_users[game1] & game_users[game2]
    if len(common_users) >= 5:
        union_users = game_users[game1] | game_users[game2]
        jaccard = len(common_users) / len(union_users)
        item_similarities[(game1,game2)] = jaccard
        item_similarities[(game2,game1)] = jaccard

print(f"Found {len(item_similarities)} item pairs with similarity ≥5 common users")

Found 18580 item pairs with similarity ≥5 common users


In [23]:
# Save to pickle file
with open('game_similarities.pkl', 'wb') as f:
    pickle.dump(item_similarities, f)

In [24]:
# Load similarities from pickle file
with open('game_similarities.pkl', 'rb') as f:
    item_similarities = pickle.load(f)

In [21]:
# Compute mean recommendation rate for each game (fraction of users who recommended it)
game_mean = {}

for game, users in game2user.items():
    ratings = [usergame2rating.get((user, game), 0) for user in users]
    count = len(ratings)
    if count > 0:
        game_mean[game] = sum(ratings) / count
    else:
        game_mean[game] = 0.0  # Or np.nan if you prefer

In [105]:
global_avg = sum(usergame2rating.values())/len(usergame2rating)

In [23]:
# Save to pickle file
with open('game_mean.pkl', 'wb') as f:
    pickle.dump(game_mean, f)

In [12]:
# Load similarities from pickle file
with open('game_mean.pkl', 'rb') as f:
    game_mean = pickle.load(f)

In [ ]:
def predict_rating(user, target_game, k=25, threshold=0.9):
    '''
    Predicts whether a user would recommend a target game (1) or not (0).
    
    :param user: User ID to predict for
    :param target_game: Game ID to predict recommendation for
    :param k: Number of similar games to consider
    :param threshold: Decision boundary for classification (default=0.5)
    :return: 1 (recommend) or 0 (not recommend)
    '''
    # Precomputed values (calculate these once outside the function)
    global_avg = sum(usergame2rating.values())/len(usergame2rating)  # Precompute this!
    
    # Get games the user has explicitly rated (0 or 1)
    rated_games = user2game.get(user, [])
    
    # Get target game's recommendation rate from precomputed dict
    game_popularity = game_mean.get(target_game, global_avg)
    
    # Collect similarities to rated games
    similarities = []
    for game in rated_games:
        # Check both similarity directions (A,B) and (B,A)
        if (target_game, game) in item_similarities:
            similarity = item_similarities[(target_game, game)]
        elif (game, target_game) in item_similarities:
            similarity = item_similarities[(game, target_game)]
        else:
            similarity = 0
        similarities.append((game, similarity))
    
    # Sort by absolute similarity and select top k neighbors
    similarities.sort(key=lambda x: abs(x[1]), reverse=True)
    neighbors = similarities[:k]
    
    # Calculate weighted deviation from user's preferences
    numerator = 0
    denominator = 0
    for game, similarity in neighbors:
        # Get user's explicit binary rating (0 or 1)
        user_rating = usergame2rating.get((user, game), 0)
        
        # Get precomputed game mean (fallback to global average)
        game_mean_val = game_mean.get(game, global_avg)
        
        # Deviation represents how much user differs from average
        deviation = user_rating - game_mean_val
        numerator += similarity * deviation
        denominator += abs(similarity)
    
    # Calculate final score
    if denominator == 0:
        score = game_popularity
        # prediction = game_popularity
    else:
        score = game_popularity + (numerator / denominator)
        # prediction = game_popularity + (numerator / denominator)


    return score
    
    # # Clamp prediction between 0 and 1
    # prediction = max(0.0, min(1.0, prediction))
    
    # # Return binary recommendation based on threshold
    # return 1 if prediction >= threshold else 0


In [48]:
target_user = 76561198300270231

In [49]:
all_games = set(game2user.keys())
rated_by_user = set(user2game[target_user])
candidate_games = all_games.difference(rated_by_user)

In [50]:
predictions = {}
for ind, game in enumerate(candidate_games):
    if ind % 100 == 0 and ind != 0:
        print("{} games processed".format(ind + 1))
    predictions[game] = predict_rating(target_user, game, k=25)

101 games processed
201 games processed
301 games processed
401 games processed
501 games processed
601 games processed
701 games processed
801 games processed
901 games processed
1001 games processed
1101 games processed
1201 games processed
1301 games processed
1401 games processed
1501 games processed
1601 games processed
1701 games processed
1801 games processed
1901 games processed
2001 games processed
2101 games processed
2201 games processed
2301 games processed
2401 games processed
2501 games processed
2601 games processed
2701 games processed
2801 games processed
2901 games processed
3001 games processed
3101 games processed
3201 games processed
3301 games processed
3401 games processed
3501 games processed
3601 games processed
3701 games processed
3801 games processed
3901 games processed
4001 games processed
4101 games processed
4201 games processed
4301 games processed
4401 games processed
4501 games processed
4601 games processed
4701 games processed
4801 games processed
4

In [51]:
top_5_recommendations = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:5]

In [52]:
top_5_recommendations

[(1304930, 1.1563839997790406),
 (990080, 1.1159832240841472),
 (1191210, 1.0919158361018826),
 (1263370, 1.0800110741971207),
 (1149660, 1.0795348837209302)]

In [53]:
print('Top 5 Recommendations for {}:'.format(target_user))
for game, pred in top_5_recommendations:
    print('Game: {}, Predicted Rating: {}'.format(game, np.round(pred, 3)))

Top 5 Recommendations for 76561198300270231:
Game: 1304930, Predicted Rating: 1.156
Game: 990080, Predicted Rating: 1.116
Game: 1191210, Predicted Rating: 1.092
Game: 1263370, Predicted Rating: 1.08
Game: 1149660, Predicted Rating: 1.08


## New User

In [60]:
def predict_rating_new_user(target_game, liked_games, disliked_games, k=25, threshold=0.5):
    '''
    Predicts whether a new user (with provided liked/disliked games) would recommend a target game.

    :param target_game: Game ID to predict recommendation for
    :param liked_games: list of appids the new user recommends (1)
    :param disliked_games: list of appids the new user does NOT recommend (0)
    :param k: Number of similar games to consider
    :param threshold: Decision boundary for classification (default=0.5)
    :return: 1 (recommend) or 0 (not recommend)
    '''
    # Precomputed values (should be calculated once outside the function)
    global_avg = sum(usergame2rating.values()) / len(usergame2rating)
    game_popularity = game_mean.get(target_game, global_avg)

    # Build the new user's pseudo-rating dictionary
    pseudo_ratings = {appid: 1 for appid in liked_games}
    pseudo_ratings.update({appid: 0 for appid in disliked_games})

    # Collect similarities to rated games
    similarities = []
    for game in liked_games + disliked_games:
        similarity = item_similarities.get((target_game, game), item_similarities.get((game, target_game), 0))
        similarities.append((game, similarity))

    # Sort by absolute similarity and select top k neighbors
    similarities.sort(key=lambda x: abs(x[1]), reverse=True)
    neighbors = similarities[:k]

    # Calculate weighted deviation from user's preferences
    numerator = 0
    denominator = 0
    for game, similarity in neighbors:
        user_rating = pseudo_ratings[game]
        game_mean_val = game_mean.get(game, global_avg)
        deviation = user_rating - game_mean_val
        numerator += similarity * deviation
        denominator += abs(similarity)

    # Calculate final prediction
    if denominator == 0:
        # prediction = game_popularity
        score = game_popularity
    else:
        score = game_popularity + (numerator / denominator)
        # prediction = game_popularity + (numerator / denominator)

    return score

    # Clamp prediction between 0 and 1
    # prediction = max(0.0, min(1.0, prediction))

    # Return binary recommendation based on threshold
    # return 1 if prediction >= threshold else 0


In [55]:
# Example usage:
recommended = [10]
not_recommended = [1652890]

In [56]:
rated_by_new_user = set(recommended + not_recommended)
candidate_games_new_user = all_games - rated_by_new_user

In [ ]:
new_user_predictions = {}
for ind, game in enumerate(candidate_games_new_user):
    if ind % 100 == 0 and ind != 0:
        print("{} games processed".format(ind + 1))
    new_user_predictions[game] = predict_rating_new_user(target_game=game, liked_games=recommended, disliked_games=not_recommended, k=25)

In [ ]:
new_user_predictions

In [72]:
top_5_recommendations_new_user = sorted(new_user_predictions.items(), key=lambda x: x[1], reverse=True)[:5]

In [73]:
top_5_recommendations_new_user

[(524290, 1.0), (393220, 1.0), (1245200, 1.0), (1114130, 1.0), (655380, 1.0)]

In [74]:
print('Top 5 Recommendations for new user: ')
for game, pred in top_5_recommendations_new_user:
    print('Game: {}, Predicted Rating: {}'.format(game, np.round(pred, 3)))

Top 5 Recommendations for new user: 
Game: 524290, Predicted Rating: 1.0
Game: 393220, Predicted Rating: 1.0
Game: 1245200, Predicted Rating: 1.0
Game: 1114130, Predicted Rating: 1.0
Game: 655380, Predicted Rating: 1.0


## Item Based Collaborative Filtering with Hyperparameter Tuning and Ranking Metrics

In [5]:
# Create dictionary mapping user_id to review count
user_review_counts = games_users_df['author_steamid'].value_counts().to_dict()

In [6]:
user_review_counts

{76561198300270231: 714,
 76561197997740965: 615,
 76561198272374716: 594,
 76561198020487283: 529,
 76561198276634938: 398,
 76561197973230221: 338,
 76561198273892969: 321,
 76561198375029180: 281,
 76561198020313592: 248,
 76561198091720018: 235,
 76561197986360343: 207,
 76561198085891656: 200,
 76561198055553431: 198,
 76561198125580541: 192,
 76561198014348460: 188,
 76561198071375554: 171,
 76561198139328230: 170,
 76561198067597141: 169,
 76561198118513291: 158,
 76561198049719840: 155,
 76561198320988354: 155,
 76561197992694498: 154,
 76561198160755391: 150,
 76561198137855251: 142,
 76561198118421272: 141,
 76561198015849657: 132,
 76561199107888635: 122,
 76561197982273259: 119,
 76561197965215862: 119,
 76561198142247081: 112,
 76561198272690093: 112,
 76561198204043497: 111,
 76561198282600262: 110,
 76561198190904849: 110,
 76561198087794419: 109,
 76561198085774720: 108,
 76561198032788858: 108,
 76561197999732661: 107,
 76561198215754123: 105,
 76561198004496223: 105,


In [16]:
# Count users with exactly 1 review
single_review_users = sum(1 for count in user_review_counts.values() if count == 1)
print(f"Number of users with exactly 1 review: {single_review_users}")

Number of users with exactly 1 review: 331327


In [60]:
review_count = 50
# Count users with exactly 1 review
single_review_users = sum(1 for count in user_review_counts.values() if count >= review_count)
print(f"Number of users with greater than or equal to {count} ratings: {single_review_users}")

Number of users with greater than or equal to 1 ratings: 125


In [61]:
num_of_games = 50
# Get users who rated more than 18 games
active_users = {user_id: count for user_id, count in user_review_counts.items() if count >= num_of_games}

# Print the number of users with more than 18 ratings
print(f"Found {len(active_users)} users who rated more than {num_of_games} games")

# Display some examples (limit to first 5)
print("\nExample active users:")
for user_id, count in list(active_users.items())[:5]:
    print(f"User {user_id}: {count} ratings")

Found 125 users who rated more than 50 games

Example active users:
User 76561198300270231: 714 ratings
User 76561197997740965: 615 ratings
User 76561198272374716: 594 ratings
User 76561198020487283: 529 ratings
User 76561198276634938: 398 ratings


In [62]:
# Calculate total ratings from active users
total_active_ratings = sum(active_users.values())

# Calculate statistics
avg_ratings_per_active_user = total_active_ratings / len(active_users)

# Calculate what percentage of all ratings come from these active users
total_ratings = sum(user_review_counts.values())
active_ratings_percentage = (total_active_ratings / total_ratings) * 100

# Print summary statistics
print(f"Active users summary:")
print(f"- Number of active users: {len(active_users)}")
print(f"- Total ratings from active users: {total_active_ratings}")
print(f"- Average ratings per active user: {avg_ratings_per_active_user:.2f}")
print(f"- Active users are {len(active_users) / len(user_review_counts):.2%} of total users")
print(f"- But provide {active_ratings_percentage:.2f}% of all ratings")

Active users summary:
- Number of active users: 125
- Total ratings from active users: 14463
- Average ratings per active user: 115.70
- Active users are 0.03% of total users
- But provide 2.90% of all ratings


In [63]:
# Optional: Get the list of user IDs only
active_user_ids = list(active_users.keys())

In [64]:
# Filter the main DataFrame to only include active users
active_users_df = games_users_df[games_users_df['author_steamid'].isin(active_user_ids)]

In [65]:
active_users_df

,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
26,142928432,10,Counter-Strike,76561198276634938,10154,1200,3233,0,3233,1642004471,...,1,103,4,0.892517,0,1,0,0,1,NaN
283,76942960,10,Counter-Strike,76561197971126613,1241,86,302047,4295,95921,1698308587,...,1,88,1,0.904832,0,1,0,0,0,NaN
352,59368085,10,Counter-Strike,76561198142247081,557,155,1235,0,1235,1574894297,...,1,115,2,0.902792,0,1,0,0,0,NaN
390,47288332,10,Counter-Strike,76561198029270650,0,143,116553,0,92450,1678138614,...,1,75,0,0.870885,0,0,0,0,0,NaN
511,27351907,10,Counter-Strike,76561198273892969,0,370,51310,0,31117,1689869339,...,1,34,0,0.810997,3,1,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497863,81725010,996160,Quest for the Golden Duck,76561198272374716,12960,3364,41,0,41,1607255564,...,1,51,0,0.832846,0,0,1,0,0,NaN
497864,47799042,996280,Buttle Tank,76561198272374716,12960,3364,144,0,144,1545719782,...,1,35,0,0.806210,2,0,1,0,0,NaN
497899,82881728,996580,Spyro™ Reignited Trilogy,76561197992694498,2717,444,4096,0,821,1694299132,...,1,67,1,0.830467,6,0,0,0,0,NaN
498026,107253617,997010,Police Simulator: Patrol Officers,76561197992694498,2717,444,1412,0,1054,1688909200,...,1,64,1,0.840699,3,0,0,1,0,NaN


In [66]:
active_user_df_ratings_only = active_users_df[['appid', 'author_steamid', 'voted_up']].copy()

In [68]:
active_user_df_ratings_only.reset_index()

,index,appid,author_steamid,voted_up
0,26,10,76561198276634938,1
1,283,10,76561197971126613,1
2,352,10,76561198142247081,1
3,390,10,76561198029270650,1
4,511,10,76561198273892969,1
...,...,...,...,...
14458,497863,996160,76561198272374716,1
14459,497864,996280,76561198272374716,1
14460,497899,996580,76561197992694498,1
14461,498026,997010,76561197992694498,1


In [69]:
active_user_df_ratings_only['appid'].nunique()

7694

In [70]:
active_user_df_ratings_only['author_steamid'].nunique()

125

#### Split Data into Training and Test Datasets

In [71]:
def train_test_split_by_user(ratings_df, min_ratings, test_count):
    """
    train_test_split_by_user is a function that splits the ratings dataframe into training and test sets on a per-user basis. Only users with 
    at least 'min_ratings' are kept. For each such user, 'test_count' ratings are randomly held out as the test set, with the remainder used as 
    training data.
    :param ratings_df: a pandas dataframe with columns user_id, game_id, and rating.
    :param min_ratings: the minimum number of ratings that the user must have so that test ratings can be held out.
    :param test_count: number of ratings held out for each eligible user.
    :return: a training dataframe and a test dataframe.
    """
    train_list = []
    test_list = []
    
    for user, group in ratings_df.groupby('author_steamid'):
        if len(group) >= min_ratings:
            group = group.sample(frac=1, random_state=42)
            test = group.iloc[:test_count]
            train = group.iloc[test_count:]
            train_list.append(train)
            test_list.append(test)
    
    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df = pd.concat(test_list).reset_index(drop=True)
    
    return train_df, test_df

In [72]:
rating_training_df, rating_test_df = train_test_split_by_user(active_user_df_ratings_only, min_ratings=18, test_count=5)

In [74]:
rating_training_df

,appid,author_steamid,voted_up
0,976730,76561197965215862,1
1,368370,76561197965215862,1
2,1117140,76561197965215862,1
3,552500,76561197965215862,1
4,403970,76561197965215862,1
...,...,...,...
13833,495420,76561199445644394,1
13834,1462040,76561199445644394,1
13835,2173800,76561199445644394,0
13836,883710,76561199445644394,1


In [75]:
rating_test_df

,appid,author_steamid,voted_up
0,304212,76561197965215862,1
1,537800,76561197965215862,1
2,1071200,76561197965215862,1
3,2378500,76561197965215862,1
4,1788370,76561197965215862,1
...,...,...,...
620,227300,76561199445644394,1
621,1220010,76561199445644394,1
622,2285570,76561199445644394,0
623,1446780,76561199445644394,1


In [77]:
rating_training_df['author_steamid'].nunique()

125

In [78]:
rating_test_df['author_steamid'].nunique()

125

#### Define Functions to Create Training and Test Dictionaries

In [79]:
def create_data_dictionaries(ratings_data_df, dataset='training'):
    """
    create_data_dictionaries is a function that creates dictionaries for user-game interactions and ratings.

    :param ratings_data_df: is a dataFrame containing 'user_id', 'game_id', and 'rating' columns.
    :param dataset: is the type of dataset ('training' or other). For 'training', user-game mappings are created; 
    otherwise, they are set as empty dictionaries.
    :return: a tuple containing user2game (dict) which maps each user_id to a list of game_ids, game2user (dict) which maps 
    each game_id to a list of user_ids, and user_game2rating (dict) which maps (user_id, game_id) pairs to their corresponding rating.
    """
    if dataset.lower() == 'training':
        user2game = ratings_data_df.groupby('author_steamid')['appid'].apply(list).to_dict()
        game2user = ratings_data_df.groupby('appid')['author_steamid'].apply(list).to_dict()
    else:
        user2game = {}
        game2user = {}
    
    user_game = zip(ratings_data_df['author_steamid'], ratings_data_df['appid'])
    user_game_rating = zip(user_game, ratings_data_df['voted_up'])
    user_game2rating = dict(user_game_rating)

    return user2game, game2user, user_game2rating

Convert Training Data to Dictionaries.

In [80]:
user2game, game2user, user_game2rating = create_data_dictionaries(
    rating_training_df
)

Convert Test Data to Dictionary

In [81]:
_, _, user_game2rating_test = create_data_dictionaries(
    rating_test_df, dataset='test'
)

In [82]:
def compute_user_average(user2game, user_game2rating):
    """
    compute_user_average is a function that calculates the average rating for each user in the dataset.
    :param user2game: a dictionary that maps each user to a list of game_ids.
    :param user_game2rating: a dictionary that maps each (user, game) pair to a rating.
    :return: a dictionary containing the average rating for each user where the key is the user and the
    value is the average rating.
    """
    user_avg = {}
    for user, games in user2game.items():
        ratings = [user_game2rating[(user, game)] for game in games]
        user_avg[user] = np.mean(ratings)
    return user_avg

In [83]:
user_avg = compute_user_average(user2game, user_game2rating)

In [85]:
def precompute_game_similarities(user2game, min_common_users=5):
    """
    Precompute Jaccard similarity for all game pairs with at least min_common_users common users.
    
    Parameters:
    -----------
    user2game : dict
        Dictionary mapping user IDs to lists of games they've rated
    user_game2rating : dict
        Dictionary mapping (user_id, game_id) pairs to ratings
    user_avg : dict
        Dictionary of average ratings for each user
    min_common_users : int, default=5
        Minimum number of common users required to compute similarity
    
    Returns:
    --------
    dict
        Dictionary with (game1_id, game2_id) tuples as keys and similarity scores as values
    """
    from itertools import combinations
    from tqdm.notebook import tqdm
    
    # First create a game_users dictionary by inverting the user2game mapping
    game_users = {}
    for user, games in user2game.items():
        for game in games:
            if game not in game_users:
                game_users[game] = set()
            game_users[game].add(user)
    
    # Now compute similarities between all game pairs
    item_similarities = {}
    game_ids = list(game_users.keys())
    total_pairs = len(game_ids) * (len(game_ids) - 1) // 2
    
    # Use tqdm for progress tracking
    for (game1, game2) in tqdm(combinations(game_ids, 2), total=total_pairs, desc="Computing similarities"):
        # Get sets of users who rated each game
        users_game1 = game_users[game1]
        users_game2 = game_users[game2]
        
        # Quick check for minimum common users before full calculation
        common_users = users_game1 & users_game2

        if len(common_users) >= min_common_users:
            # Calculate Jaccard similarity
            union_users = users_game1 | users_game2
            jaccard = len(common_users) / len(union_users)
            # Store similarity in both directions for easy lookup
            item_similarities[(game1, game2)] = jaccard
            item_similarities[(game2, game1)] = jaccard
        else:
            item_similarities[(game1, game2)] = 0.0
            item_similarities[(game2, game1)] = 0.0
    
    print(f"Found {len(item_similarities) // 2} unique game pairs with ≥{min_common_users} common users")
    return item_similarities

In [86]:
min_common_value = 5

In [87]:
similarity_matrix = precompute_game_similarities(user2game, min_common_value)

Computing similarities:   0%|          | 0/28038816 [00:00<?, ?it/s]

Found 28038816 unique game pairs with ≥5 common users


In [88]:
with open('item-item-similarity.json', 'wb') as f:
    pickle.dump(similarity_matrix, f)

In [42]:
with open('item-item-similarity.json', 'rb') as f:
    similarity_matrix = pickle.load(f)

In [90]:
# Calculate global average
global_avg = sum(user_game2rating.values()) / len(user_game2rating) if user_game2rating else 0.5

In [91]:
# Compute mean recommendation rate for each game
game_mean = {}
for game, users in game2user.items():
    ratings = [user_game2rating.get((user, game), 0) for user in users]
    count = len(ratings)
    if count > 0:
        game_mean[game] = sum(ratings) / count
    else:
        game_mean[game] = 0.0

In [92]:
def predict_rating(user, target_game, user2game, user_game2rating, k_neighbors, similarity_matrix):
    '''
    predict_rating is a function that predicts the rating that user "user" would give item "target_movie" using 
    k_neighbors other items that user "user" has rated.

    :param user: User ID to predict for
    :param target_game: Game ID to predict recommendation for
    :param k_neighbors: Number of similar games to consider
    '''
    
    # Get games the user has explicitly rated (0 or 1)
    rated_games = user2game.get(user, [])
    
    # Get target game's recommendation rate from precomputed dict
    game_popularity = game_mean.get(target_game, global_avg)    
    
    # Collect similarities to rated games
    similarities = []
    for game in rated_games:
        # Check both similarity directions (A,B) and (B,A)
        if (target_game, game) in similarity_matrix:
            similarity = similarity_matrix[(target_game, game)]
        elif (game, target_game) in similarity_matrix:
            similarity = similarity_matrix[(game, target_game)]
        else:
            similarity = 0
        similarities.append((game, similarity))
    
    # Sort by absolute similarity and select top k neighbors
    similarities.sort(key=lambda x: abs(x[1]), reverse=True)
    neighbors = similarities[:k_neighbors]
    
    # Calculate weighted deviation from user's preferences
    numerator = 0
    denominator = 0
    for game, similarity in neighbors:
        # Get user's explicit binary rating (0 or 1)
        user_rating = user_game2rating.get((user, game), 0)
        
        # Get precomputed game mean (fallback to global average)
        game_mean_val = game_mean.get(game, global_avg)
        
        # Deviation represents how much user differs from average
        deviation = user_rating - game_mean_val
        numerator += similarity * deviation
        denominator += abs(similarity)
    
    # Calculate final score
    if denominator == 0:
        score = game_popularity
        # prediction = game_popularity
    else:
        score = game_popularity + (numerator / denominator)
        # prediction = game_popularity + (numerator / denominator)

    return score

In [93]:
def evaluate_ranking_metrics(user2game, user_game2rating, similarity_matrix, dataset_rating, k_neighbors, k_items):
    """
    :param user2game: a dictionary that maps each user to a list of game_ids.
    :param user_game2rating: a dictionary that maps each (user, game) pair to a rating.
    :param user_avg: a dictionary of the average ratings for each user.
    :param similarity_matrix: the precomputed matrix of similarity scores between all pairs of users.
    :param dataset_rating: a dictionary containing a mapping from (user, game) pair to a rating.
    :param k_neighbors: the number of nearest neighbors to be considered when predicting the user's rating.
    :param k_items: the number of top k items for evaluation.
    :return: the computed Precision@k_items, Recall@k_items, and F1@k_items for that user.
    """

    all_games = {game for (user, game), rating in dataset_rating.items()}
    
    user2games_relevant = {}
    for (user, game), rating in dataset_rating.items():
        # if rating >= relevance_threshold:
        if user not in user2games_relevant:
            user2games_relevant[user] = set()
        user2games_relevant[user].add(game)
    
    metrics_list = []
    
    for user, ground_truth in user2games_relevant.items():
    
        training_games = set(user2game[user])
        candidate_games = list(all_games - training_games)

        predictions = []
        for game in candidate_games:
            pred_rating = predict_rating(user, game, user2game, user_game2rating, 
                                             k_neighbors, similarity_matrix)
            predictions.append((game, pred_rating))

        predictions_sorted = sorted(predictions, key=lambda x: x[1], reverse=True)
        top_k_games = [game for game, score in predictions_sorted[:k_items]]
        
        # calculate metrics
        num_relevant_in_top_k = len(set(top_k_games) & ground_truth)
        precision = num_relevant_in_top_k / k_items
        recall = num_relevant_in_top_k / len(ground_truth) if ground_truth else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics_list.append((precision, recall, f1))

    if not metrics_list:
        return 0, 0, 0
    
    avg_precision = np.mean([m[0] for m in metrics_list])
    avg_recall = np.mean([m[1] for m in metrics_list])
    avg_f1 = np.mean([m[2] for m in metrics_list])
    
    return avg_precision, avg_recall, avg_f1

In [94]:
k_items = 5

In [ ]:
k_neighbors = 25

In [96]:
avg_precision, avg_recall, avg_f1 = evaluate_ranking_metrics(
    user2game,
    user_game2rating,
    similarity_matrix,
    user_game2rating_test,
    k_neighbors,
    k_items
)

print('Precision@{}: {}, Recall@{}: {}, F1@{}: {}'.format(
    k_items, np.round(avg_precision, 3), k_items, np.round(avg_recall, 3), k_items, np.round(avg_f1, 3))
)

Precision@5: 0.026, Recall@5: 0.026, F1@5: 0.026


In [100]:
k_items = 10

In [101]:
candidate_k_neighbors = [10, 20, 25, 30, 35]

results = {}
for k in candidate_k_neighbors:
    avg_precision, avg_recall, avg_f1 = evaluate_ranking_metrics(
        user2game,
        user_game2rating,
        similarity_matrix,
        user_game2rating_test,
        k,
        k_items,
    )
    
    results[k] = {
        'Precision@{}'.format(k_items): avg_precision,
        'Recall@{}'.format(k_items): avg_recall,
        'F1@{}'.format(k_items): avg_f1
    }

    print(
        'Precision@{}: {}, Recall@{}: {}, F1@{}: {}'.format(
            k_items, np.round(avg_precision, 3), k_items, np.round(avg_recall, 3), k_items, np.round(avg_f1, 3)
        )
    )

Precision@10: 0.017, Recall@10: 0.034, F1@10: 0.022
Precision@10: 0.017, Recall@10: 0.034, F1@10: 0.022
Precision@10: 0.017, Recall@10: 0.034, F1@10: 0.022
Precision@10: 0.017, Recall@10: 0.034, F1@10: 0.022
Precision@10: 0.017, Recall@10: 0.034, F1@10: 0.022


## Creating a Test Case

In [58]:
# Count the number of pairs with similarity scores greater than 0
positive_similarity_count = sum(1 for score in similarity_matrix.values() if score > 0)

print(f"Number of pairs with similarity scores > 0: {positive_similarity_count}")

Number of pairs with similarity scores > 0: 4888


In [59]:
# Get all pairs with similarity scores greater than 0
positive_similarity_pairs = [(pair, score) for pair, score in similarity_matrix.items() if score > 0]

# Display the first 5 pairs
print("First 5 pairs with similarity scores > 0:")
for pair, score in positive_similarity_pairs[:5]:
    print(f"Pair: {pair}, Similarity Score: {score}")

First 5 pairs with similarity scores > 0:
Pair: (1145360, 524220), Similarity Score: 0.12280701754385964
Pair: (524220, 1145360), Similarity Score: 0.12280701754385964
Pair: (1145360, 1466640), Similarity Score: 0.12195121951219512
Pair: (1466640, 1145360), Similarity Score: 0.12195121951219512
Pair: (1145360, 270880), Similarity Score: 0.1111111111111111


In [ ]:
def predict_rating_new_user(target_game, liked_games, disliked_games, k=25, threshold=0.5):
    '''
    Predicts whether a new user (with provided liked/disliked games) would recommend a target game.

    :param target_game: Game ID to predict recommendation for
    :param liked_games: list of appids the new user recommends (1)
    :param disliked_games: list of appids the new user does NOT recommend (0)
    :param k: Number of similar games to consider
    :param threshold: Decision boundary for classification (default=0.5)
    :return: 1 (recommend) or 0 (not recommend)
    '''
    # Precomputed values (should be calculated once outside the function)
    global_avg = sum(usergame2rating.values()) / len(usergame2rating)
    game_popularity = game_mean.get(target_game, global_avg)

    # Build the new user's pseudo-rating dictionary
    pseudo_ratings = {appid: 1 for appid in liked_games}
    pseudo_ratings.update({appid: 0 for appid in disliked_games})

    # Collect similarities to rated games
    similarities = []
    for game in liked_games + disliked_games:
        similarity = similarity_matrix.get((target_game, game), similarity_matrix.get((game, target_game), 0))
        similarities.append((game, similarity))

    # Sort by absolute similarity and select top k neighbors
    similarities.sort(key=lambda x: abs(x[1]), reverse=True)
    neighbors = similarities[:k]

    # Calculate weighted deviation from user's preferences
    numerator = 0
    denominator = 0
    for game, similarity in neighbors:
        user_rating = pseudo_ratings[game]
        game_mean_val = game_mean.get(game, global_avg)
        deviation = user_rating - game_mean_val
        numerator += similarity * deviation
        denominator += abs(similarity)

    # Calculate final prediction
    if denominator == 0:
        # prediction = game_popularity
        score = game_popularity
    else:
        score = game_popularity + (numerator / denominator)
        # prediction = game_popularity + (numerator / denominator)

    return score

    # Clamp prediction between 0 and 1
    # prediction = max(0.0, min(1.0, prediction))

    # Return binary recommendation based on threshold
    # return 1 if prediction >= threshold else 0
